In [69]:
#pip install request
import pandas as pd
import numpy as np
import re
import urllib
import json
import csv
import pprint

In [70]:
ip_list = []
filt = []
geo_url = "http://freegeoip.net/json/"
rdap_url = "http://rdap.apnic.net/ip/"

In [71]:
def parsefile(textfile):
    with open(textfile, "r") as txt:
        content = txt.read()
        ips = re.findall(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", content)
        for i in ips:
            ip_list.append(i)
    with open('IP_List.csv', 'wb') as csvfile:
        geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        geo_write.writerow(ip_list)

    print textfile+" has been parsed into file and array"

In [72]:
def geoIP_search(string):
    result = []
    res = urllib.urlopen(geo_url+string)
    data = json.loads(res.read())
    ip = data['ip']
    country = data['country_code']
    region = data['region_code']
    regname = data['region_name']
    zipc = data['zip_code']
    longi = str(data['longitude'])
    lati = str(data['latitude'])
    result.append(ip.encode('utf-8'))
    result.append(country.encode('utf-8'))
    result.append(region.encode('utf-8'))
    result.append(regname.encode('utf-8'))
    result.append(zipc.encode('utf-8'))
    result.append(longi.encode('utf-8'))
    result.append(lati.encode('utf-8'))
    name = raw_input("input csv file name")
    with open(name+'.csv', 'wb') as csvfile:
        geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        geo_write.writerow(result)
    with open(name+'.csv', 'rb') as csvfile:
        geo_reader = csv.reader(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        for i in geo_reader:
            data = np.array([i])
            df = pd.DataFrame(data, columns=["ip","country","region","area","zip","longitude","latitude"])
    return df

In [73]:
def geoIP_all(array):
    geo_list = []
    for i in array:
        result = []
        res = urllib.urlopen(geo_url+i)
        data = json.loads(res.read())
        ip = data['ip']
        country = data['country_code']
        region = data['region_code']
        regname = data['region_name']
        zipc = data['zip_code']
        longi = str(data['longitude'])
        lati = str(data['latitude']) 
        result.append(ip.encode('utf-8'))
        result.append(country.encode('utf-8'))
        result.append(region.encode('utf-8'))
        result.append(regname.encode('utf-8'))
        result.append(zipc.encode('utf-8'))
        result.append(longi.encode('utf-8'))
        result.append(lati.encode('utf-8'))
        geo_list.append(result)
    with open('geoIPAll.csv', 'wb') as csvfile:
        geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        geo_write.writerow(geo_list)
    with open('geoIPAll.csv', 'rb') as csvfile:
        geo_reader = csv.reader(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        for i in geo_reader:
            data = np.array(i)
            df = pd.DataFrame(geo_list, columns=["ip","country","region","area","zip","longitude","latitude"])
    return df

In [74]:
def rdap_search(string):
    res = urllib.urlopen(rdap_url+string)
    data = json.loads(res.read())
    #ip
    try:
        nhandle = (data['handle']).encode('utf-8')
    except (KeyError, IndexError):
        nhandle = ''
    try:
        startip = (data['startAddress']).encode('utf-8')
    except (KeyError, IndexError):
        startip = ''
    try:
        endip = (data['endAddress']).encode('utf-8')
    except (KeyError, IndexError):
        endip = ''
    try:
        ipv = (data['ipVersion']).encode('utf-8')
    except (KeyError, IndexError):
        ipv = ''
    try:
        phandle = (data['parentHandle']).encode('utf-8')
    except (KeyError, IndexError):
        phandle = ''
    try: 
        update = (data['events'][0]['eventDate']).encode('utf-8')
    except (KeyError, IndexError):
        update = ''
    #entity
    try: 
        ename = (data['entities'][0]['handle']).encode('utf-8')
    except (KeyError, IndexError):
        ename = ''
    try: 
        name = (data['name']).encode('utf-8')
    except (KeyError, IndexError):
        name = ''
    try: 
        erole1 = (data['entities'][0]['roles'][0])
    except (KeyError, IndexError):
        erole1 = ''
    try: 
        erole2 = (data['entities'][1]['roles'][0])
    except (KeyError, IndexError):
        erole2 = ''
    try: 
        ref = (data['links'][1]['href']).encode('utf-8')
    except (KeyError, IndexError):
        ref = ''

    ipform = "IP data"+"\nHandle: "+nhandle+"\nParent Handle: "+phandle+"\nNet range: "+rang+"\nIP Version: "+ipv+"\nLast Changed: "+update
    entform = "\nEntity"+"\nHandle: "+ename+"\nName: "+name+"\nRole: "+erole1+", "+erole2
    startend = "----------------------------------------------------------------"
    form = "RDAP:"+"\n"+ipform+"\n"+entform+"\n"+"Reference: "+ref+"\n"+startend+"\n"
    print form
    name = raw_input("input csv file name")
    with open(name+'.csv', 'wb') as csvfile:
        geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        geo_write.writerow(form)

In [75]:
def rdap_all(array):
    rdap = []
    for i in array:
        res = urllib.urlopen("http://rdap.apnic.net/ip/"+i)
        data = json.loads(res.read())
        try:
            nhandle = (data['handle']).encode('utf-8')
        except (KeyError, IndexError):
            nhandle = ''
        try:
            startip = (data['startAddress']).encode('utf-8')
        except (KeyError, IndexError):
            startip = ''
        try:
            endip = (data['endAddress']).encode('utf-8')
        except (KeyError, IndexError):
            endip = ''
        try:
            ipv = (data['ipVersion']).encode('utf-8')
        except (KeyError, IndexError):
            ipv = ''
        try:
            phandle = (data['parentHandle']).encode('utf-8')
        except (KeyError, IndexError):
            phandle = ''
        try: 
            update = (data['events'][0]['eventDate']).encode('utf-8')
        except (KeyError, IndexError):
            update = ''
        #entity
        try: 
            ename = (data['entities'][0]['handle']).encode('utf-8')
        except (KeyError, IndexError):
            ename = ''
        try: 
            name = (data['name']).encode('utf-8')
        except (KeyError, IndexError):
            name = ''
        try: 
            erole1 = (data['entities'][0]['roles'][0])
        except (KeyError, IndexError):
            erole1 = ''
        try: 
            erole2 = (data['entities'][1]['roles'][0])
        except (KeyError, IndexError):
            erole2 = ''
        try: 
            ref = (data['links'][1]['href']).encode('utf-8')
        except (KeyError, IndexError):
            ref = ''

        ipform = "IP data"+"\nHandle: "+nhandle+"\nParent Handle: "+phandle+"\nNet range: "+rang+"\nIP Version: "+ipv+"\nLast Changed: "+update
        entform = "\nEntity"+"\nHandle: "+ename+"\nName: "+name+"\nRole: "+erole1+", "+erole2
        startend = "----------------------------------------------------------------"
        form = "RDAP:"+"\n"+ipform+"\n"+entform+"\n"+"Reference: "+ref+"\n"+startend+"\n"
        rdap.append(form)
    with open('RDAPAll.csv', 'wb') as csvfile:
        geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        geo_write.writerow(rdap)


In [77]:
#main
tfile = raw_input("Input txt file: ")
parsefile(tfile)
#menu
menuop = "(1) Geo IP look up of all ip addresses in file"+"\n(2) Geo IP look up of specific ip addres in file"+"\n(3) RDAP look up for all ip addresses in file"+"\n(4) RDAP look up for specific ip addres in file"
print menuop
menu = raw_input("Input menu option: ")
if (menu == "1"):
    geoIP_all(ip_list)
    menuop = "\n(2) Geo IP look up of specific ip addres in file"+"\n(3) RDAP look up for all ip addresses in file"+"\n(4) RDAP look up for specific ip addres in file"
    print menuop
    menu = raw_input("Input menu option: ")
if (menu == "2"):
    iplook = raw_input("Input Ip from file for geo look up: ")
    geoIP_search(iplook)
    menuop = "(1) Geo IP look up of all ip addresses in file"+"\n(3) RDAP look up for all ip addresses in file"+"\n(4) RDAP look up for specific ip addres in file"
    print menuop
    menu = raw_input("Input menu option: ")
if (menu == "3"):
    rdap_all(ip_list)
    menuop = "(1) Geo IP look up of all ip addresses in file"+"\n(2) Geo IP look up of specific ip addres in file"+"\n(4) RDAP look up for specific ip addres in file"
    print menuop
    menu = raw_input("Input menu option: ")
if (menu == "4"):
    rdap_look = raw_input("Input ip from file for RDAP look up: ")
    rdap_search(rdap_look)
    menuop = "(1) Geo IP look up of all ip addresses in file"+"\n(2) Geo IP look up of specific ip addres in file"
    print menuop
else:
    menuop = "(1) Geo IP look up of all ip addresses in file"+"\n(2) Geo IP look up of specific ip addres in file"+"\n(3) RDAP look up for all ip addresses in file"+"\n(4) RDAP look up for specific ip addres in file"
    print menuop
    menu = raw_input("Input menu option: ")

Input txt file: list_of_ips.txt
list_of_ips.txt has been parsed into file and array
(1) Geo IP look up of all ip addresses in file
(2) Geo IP look up of specific ip addres in file
(3) RDAP look up for all ip addresses in file
(4) RDAP look up for specific ip addres in file
Input menu option: 1


NameError: name 'geoip_all' is not defined

In [534]:
#test for json lib and formatting 
result = []
res = urllib.urlopen("http://freegeoip.net/json/131.204.254.109")
data = json.loads(res.read())
ip = data['ip']
country = data['country_code']
region = data['region_code']
regname = data['region_name']
zipc = data['zip_code']
longi = str(data['longitude'])
lati = str(data['latitude']) 
result.append(ip.encode('utf-8'))
result.append(country.encode('utf-8'))
result.append(region.encode('utf-8'))
result.append(regname.encode('utf-8'))
result.append(zipc.encode('utf-8'))
result.append(longi.encode('utf-8'))
result.append(lati.encode('utf-8'))
name = raw_input("input csv file name")
with open(name+'.csv', 'wb') as csvfile:
    geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    geo_write.writerow(result)
with open(name+'.csv', 'rb') as csvfile:
    geo_reader = csv.reader(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    for i in geo_reader:
        data = np.array([i])
        df = pd.DataFrame(data, columns=["ip","country","region","area","zip","longitude","latitude"])
df

input csv file name1


,ip,country,region,area,zip,longitude,latitude
0,131.204.254.109,US,AL,Alabama,36849,-85.487,32.602


In [597]:
arr = []
lista =['81.44.150.240', '131.204.254.109']
#headers = ["ip","country","region","area","zip","longitude","latitude"]
#arr.append(headers)
    
for i in lista:
    result = []
    res = urllib.urlopen(geo_url+i)
    data = json.loads(res.read())
    ip = data['ip']
    country = data['country_code']
    region = data['region_code']
    regname = data['region_name']
    zipc = data['zip_code']
    longi = str(data['longitude'])
    lati = str(data['latitude']) 
    result.append(ip.encode('utf-8'))
    result.append(country.encode('utf-8'))
    result.append(region.encode('utf-8'))
    result.append(regname.encode('utf-8'))
    result.append(zipc.encode('utf-8'))
    result.append(longi.encode('utf-8'))
    result.append(lati.encode('utf-8'))
    arr.append(result)
with open('all.csv', 'wb') as csvfile:
    geo_write = csv.writer(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    for i in arr:
        geo_write.writerow(i)

with open('all.csv', 'rb') as csvfile:
    geo_reader = csv.reader(csvfile, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    for i in geo_reader:
        data = np.array(i)
        df = pd.DataFrame(arr, columns=["ip","country","region","area","zip","longitude","latitude"])
df


,ip,country,region,area,zip,longitude,latitude
0,81.44.150.240,ES,GA,Galicia,36472,-8.5668,42.1167
1,131.204.254.109,US,AL,Alabama,36849,-85.487,32.602


In [49]:
lista =['131.204.254.109', '244.36.171.60']
for i in lista:    
    res = urllib.urlopen("http://rdap.arin.net/registry/ip/"+i)
    data = json.loads(res.read())
    #error checking
    #ip
    
    try:
        nhandle = (data['handle']).encode('utf-8')
    except (KeyError, IndexError):
        nhandle = ''
    try:
        startip = (data['startAddress']).encode('utf-8')
    except (KeyError, IndexError):
        startip = ''
    try:
        endip = (data['endAddress']).encode('utf-8')
    except (KeyError, IndexError):
        endip = ''
    try:
        ipv = (data['ipVersion']).encode('utf-8')
    except (KeyError, IndexError):
        ipv = ''
    try:
        phandle = (data['parentHandle']).encode('utf-8')
    except (KeyError, IndexError):
        phandle = ''
    try: 
        update = (data['events'][0]['eventDate']).encode('utf-8')
    except (KeyError, IndexError):
        update = ''
    #entity
    try: 
        ename = (data['entities'][0]['handle']).encode('utf-8')
    except (KeyError, IndexError):
        ename = ''
    try: 
        name = (data['name']).encode('utf-8')
    except (KeyError, IndexError):
        name = ''
    try: 
        erole1 = (data['entities'][0]['roles'][0])
    except (KeyError, IndexError):
        erole1 = ''
    try: 
        erole2 = (data['entities'][1]['roles'][0])
    except (KeyError, IndexError):
        erole2 = ''
    try: 
        ref = (data['links'][1]['href']).encode('utf-8')
    except (KeyError, IndexError):
        ref = ''

    ipform = "IP data"+"\nHandle: "+nhandle+"\nParent Handle: "+phandle+"\nNet range: "+rang+"\nIP Version: "+ipv+"\nLast Changed: "+update
    entform = "\nEntity"+"\nHandle: "+ename+"\nName: "+name+"\nRole: "+erole1+", "+erole2
    startend = "----------------------------------------------------------------"
    form = "RDAP:"+"\n"+ipform+"\n"+entform+"\n"+"Reference: "+ref+"\n"+startend+"\n"
    print form

RDAP:
IP data
Handle: NET-131-204-0-0-1
Parent Handle: NET-131-0-0-0-0
Net range: 240.0.0.0 - 255.255.255.255
IP Version: v4
Last Changed: 2013-02-21T18:06:41-05:00

Entity
Handle: AUBURN
Name: AU-NET
Role: registrant, technical
Reference: https://whois.arin.net/rest/net/NET-131-204-0-0-1
----------------------------------------------------------------

RDAP:
IP data
Handle: NET-240-0-0-0-0
Parent Handle: 
Net range: 240.0.0.0 - 255.255.255.255
IP Version: v4
Last Changed: 2013-08-30T13:39:06-04:00

Entity
Handle: IANA
Name: SPECIAL-IPV4-FUTURE-USE-IANA-RESERVED
Role: registrant, 
Reference: https://whois.arin.net/rest/net/NET-240-0-0-0-0
----------------------------------------------------------------



In [41]:
res = urllib.urlopen("http://rdap.apnic.net/ip/244.36.171.60")
data = json.loads(res.read())
#error checking
#ip
if (data['errorCode']):
    form = 'no data on ip address'
    print form
else:
    try:
        nhandle = (data['handle']).encode('utf-8')
    except (KeyError, IndexError):
        nhandle = ''
    try:
        startip = (data['startAddress']).encode('utf-8')
    except (KeyError, IndexError):
        startip = ''
    try:
        endip = (data['endAddress']).encode('utf-8')
    except (KeyError, IndexError):
        endip = ''
    try:
        ipv = (data['ipVersion']).encode('utf-8')
    except (KeyError, IndexError):
        ipv = ''
    try:
        phandle = (data['parentHandle']).encode('utf-8')
    except (KeyError, IndexError):
        phandle = ''
    try: 
        update = (data['events'][0]['eventDate']).encode('utf-8')
    except (KeyError, IndexError):
        update = ''
    rang = startip+" - "+endip
    #entity
    try: 
        ename = (data['entities'][0]['handle']).encode('utf-8')
    except (KeyError, IndexError):
        ename = ''
    try: 
        name = (data['name']).encode('utf-8')
    except (KeyError, IndexError):
        name = ''
    try: 
        erole1 = (data['entities'][0]['roles'][0])
    except (KeyError, IndexError):
        erole1 = ''
    try: 
        erole2 = (data['entities'][1]['roles'][0])
    except (KeyError, IndexError):
        erole2 = ''
    try: 
        ref = (data['links'][1]['href']).encode('utf-8')
    except (KeyError, IndexError):
        ref = ''

    ipform = "IP data"+"\nHandle: "+nhandle+"\nParent Handle: "+phandle+"\nNet range: "+rang+"\nIP Version: "+ipv+"\nLast Changed: "+update
    entform = "\nEntity"+"\nHandle: "+ename+"\nName: "+name+"\nRole: "+erole1+erole2
    startend = "----------------------------------------------------------------"
    form = "RDAP:"+"\n"+ipform+"\n"+entform+"\n"+"Reference: "+ref+"\n"+startend+"\n"
    print form

no data on ip address


In [67]:
li = ['244.36.171.60', '131.204.254.109']
geo_list = []
country = raw_input('input country filter: ')
for i in li: 
    result = []
    res = urllib.urlopen(geo_url+i)
    data = json.loads(res.read())
    ip = data['ip']
    country = data['country_code']
    region = data['region_code']
    regname = data['region_name']
    zipc = data['zip_code']
    longi = str(data['longitude'])
    lati = str(data['latitude'])
    result.append(ip.encode('utf-8'))
    result.append(country.encode('utf-8'))
    result.append(region.encode('utf-8'))
    result.append(regname.encode('utf-8'))
    result.append(zipc.encode('utf-8'))
    result.append(longi.encode('utf-8'))
    result.append(lati.encode('utf-8'))
    geo_list.append(result)
    if (country == filt):
        geo_list.append(result)
    
print geo_list

input country filter: 
[['244.36.171.60', '', '', '', '', '0', '0'], ['131.204.254.109', 'US', 'AL', 'Alabama', '36849', '-85.487', '32.602']]
